In [3]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 50000/50000 [00:07<00:00, 6459.72 examples/s]


In [5]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [25]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [11]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [15]:
trainer.train()

                                                 
 33%|███▎      | 125/375 [03:10<04:34,  1.10s/it]

{'eval_loss': 1.3332122564315796, 'eval_accuracy': 0.397, 'eval_runtime': 37.3109, 'eval_samples_per_second': 26.802, 'eval_steps_per_second': 3.35, 'epoch': 1.0}


                                                 
 67%|██████▋   | 250/375 [06:07<02:21,  1.13s/it]

{'eval_loss': 1.128957986831665, 'eval_accuracy': 0.495, 'eval_runtime': 36.893, 'eval_samples_per_second': 27.105, 'eval_steps_per_second': 3.388, 'epoch': 2.0}


                                                 
100%|██████████| 375/375 [09:11<00:00,  1.47s/it]

{'eval_loss': 1.0302726030349731, 'eval_accuracy': 0.575, 'eval_runtime': 38.247, 'eval_samples_per_second': 26.146, 'eval_steps_per_second': 3.268, 'epoch': 3.0}
{'train_runtime': 551.4194, 'train_samples_per_second': 5.441, 'train_steps_per_second': 0.68, 'train_loss': 1.0850947265625, 'epoch': 3.0}


TrainOutput(global_step=375, training_loss=1.0850947265625, metrics={'train_runtime': 551.4194, 'train_samples_per_second': 5.441, 'train_steps_per_second': 0.68, 'total_flos': 789354427392000.0, 'train_loss': 1.0850947265625, 'epoch': 3.0})

In [27]:
# Evaluate the model
results = trainer.evaluate()


100%|██████████| 125/125 [00:35<00:00,  3.48it/s]


In [28]:
results

{'eval_loss': 1.0302726030349731,
 'eval_accuracy': 0.575,
 'eval_runtime': 41.0111,
 'eval_samples_per_second': 24.384,
 'eval_steps_per_second': 3.048,
 'epoch': 3.0}

In [29]:
model("Hello, this one sentence!")

TypeError: string indices must be integers